In [3]:
# Dependencies and setup 
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from splinter import Browser
import time 



In [4]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/bailey/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


# NASA Mars News

In [5]:
#splinter for browser
news_url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)

In [6]:

# HTML object
html = browser.html

# Parse HTML with BS
soup = bs(html, 'html.parser')

#Select latest category
browser.find_by_id('date').first.click()
browser.find_option_by_text('Latest').first.click()

#Select  categories
browser.find_by_id('categories').first.click()
browser.find_option_by_text('All Categories').first.click()
browser.find_by_id('categories').first.click()
time.sleep(0)

#Select latest title
results=soup.find('li', class_="slide")

# Pase the first title from the results 
news_title =results.find('div',class_='content_title').text.strip()

# Parse out the first paragraph associatd with the tile from results
news_p=results.find('div',class_='article_teaser_body').text.strip()

AttributeError: 'NoneType' object has no attribute 'find'

# JPL Mars Space Images - Featured Image

In [7]:
#splinter
images_url="https://www.jpl.nasa.gov/images?search=&category=Mars"
browser.visit(images_url)
time.sleep(1)

In [8]:
# HTML object
html = browser.html

# Parse HTML with BS
soup = bs(html, 'html.parser')
browser.find_by_id('filter_Mars').click()
time.sleep(0.5)
featured_image_url=browser.find_by_css('.BaseImage')['data-src']

In [9]:
#print(featured_image_url)

# Mars Facts

In [10]:
#splinter
facts_url="https://space-facts.com/mars/"
browser.visit(facts_url)

In [11]:
# Use Panda's `read_html` to parse the url
facts_tables = pd.read_html(facts_url)
# facts_tables

mars_0_df=facts_tables[0]
# mars_0_df

column_names={
    0: "Description",
    1: "Mars Fact"
}

mars_1_df=mars_0_df.rename(columns=column_names)
# mars_1_df

mars_df=mars_1_df.set_index("Description")
# mars_df

mars_facts_html=mars_df.to_html()


# Mars Hemispheres

In [12]:
#Splinter
hemispheres_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

In [13]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

#select hemisphere info
#image link

results = soup.find_all('div', class_='item')
results

hemisphere_image_urls=[]
count=0

for each_result in results: 
    
    title=each_result.find('h3').text

    
    browser.find_by_tag("h3")[count].click()
    time.sleep(0.5)
    
    count=count+1
    
    html = browser.html

    img_url=browser.find_by_text('Sample')['href']
    
    hemisphere_dict={
        "title": title,
        "img_url":img_url
    }
    
    hemisphere_image_urls.append(hemisphere_dict)
    
    time.sleep(0.5)
    
    browser.visit(hemispheres_url)

In [14]:
# Close browser session as good practice

browser.quit()

In [15]:
mars_dict={
    "Title":news_title,
    "Paragraph":news_p,
    "FeaturedImage": featured_image_url,
    "MarsFacts": mars_facts_html,
    "HemisphereImages": hemisphere_image_urls
}

print(mars_dict)

NameError: name 'news_title' is not defined